# Plain RSA implementation

Create a Python program that will be able to encrypt and decrypt the contents of any given file using your own RSA implementation of the encrypt and decrypt functions without padding,based on gmpy2.powmod, as shown below.

Encryption:
1. Convert the plaintext to integers.
2. c = pe mod n, where e and n are values derived from the public key.

Decryption:
1. p = cd mod n, where d and n are values derived from the private key.
2. Convert the integers back to the plaintext.

The public and private keys to be used can be given as input by the user or if any input regarding the public and private keys is missing, the program should be able to generate them using the function rsa.generate_private_key from the cryptography library and save them in the files public_key.pem and private_key.pem respectively.

Your program should support the following arguments:

● -e plaintext message file: Encrypt file contents

● -d encrypted message file: Decrypt file contents

● -pub public key file: File containing the public key

● -prv private key file: File containing the private key

● -o output file: Output file, if both encryption and decryption are specified, this file should contain both outputs. If no output file is specified the program should print on the console.


In [ ]:
import argparse
import sys
import gmpy2
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.primitives.asymmetric.rsa import generate_private_key

from cryptography.hazmat.backends import default_backend

In [ ]:
def bytes_to_int(text: bytes):
    return int.from_bytes(text, 'big')


def int_to_bytes(num):
    num_int = int(num)
    return int.to_bytes(num_int, (num_int.bit_length()+7)//8, "big")


def encrypt(plain: bytes, e: int, n: int, block_size: int | None = None):
    if block_size is None:
        block_size = (n.bit_length()+7)//8

    blocks = [bytes_to_int(plain[i:i+block_size])
              for i in range(0, len(plain), block_size)]

    return b"".join([int_to_bytes(gmpy2.powmod(block, e, n)) for block in blocks])


def decrypt(cipher: bytes, d: int, n: int, block_size: int | None = None):
    if block_size is None:
        block_size = (n.bit_length()+7)//8

    blocks = [bytes_to_int(cipher[i:i+block_size])
              for i in range(0, len(cipher), block_size)]

    return b"".join([int_to_bytes(gmpy2.powmod(block, d, n)) for block in blocks])


In [ ]:
def main(arguments):
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument(
        "-e", "--encrypt", help="Msg to be encrypted", type=str)
    parser.add_argument(
        "-d", "--decrypt", help="Msg to be decrypted", type=str)
    parser.add_argument(
        "-pub", "--public-key", help="File containing the public key", type=argparse.FileType('r'))
    parser.add_argument(
        "-prv", "--private-key", help="File containing the private key", type=argparse.FileType('r'))
    parser.add_argument(
        "-o", "--output", help="The output file", type=argparse.FileType('w'))

    args = parser.parse_args(arguments)

    if args.encrypt:
        if args.public_key:
            public_key = load_pem_public_key(
                args.public_key.read().encode())
        else:
            priv_key = generate_private_key(0x10001, 2048)
            with open("private_key.pem", "w") as priv_key_file:
                priv_key_file.write(priv_key.private_bytes(encoding=serialization.Encoding.PEM,
                                    format=serialization.PrivateFormat.PKCS8, encryption_algorithm=serialization.NoEncryption()).decode())

            public_key = priv_key.public_key()
            with open("public_key.pem", "w") as pub_key_file:
                pub_key_file.write(public_key.public_bytes(
                    encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo).decode())

        numbers = public_key.public_numbers()
        encrypted = encrypt(args.encrypt.encode(), numbers.e, numbers.n)

        if args.output:
            args.output.write(encrypted.hex())
        print(encrypted.hex())

    if args.decrypt:
        if args.private_key:
            private_key = load_pem_private_key(
                args.private_key.read().encode(), password=None)
        else:
            private_key = generate_private_key(0x10001, 2048)
            with open("private_key.pem", "w") as priv_key_file:
                priv_key_file.write(private_key.private_bytes(encoding=serialization.Encoding.PEM,
                                    format=serialization.PrivateFormat.PKCS8, encryption_algorithm=serialization.NoEncryption()).decode())

            public_key = private_key.public_key()
            with open("public_key.pem", "w") as pub_key_file:
                pub_key_file.write(public_key.public_bytes(
                    encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo).decode())

        decrypted = decrypt(bytes.fromhex(args.decrypt),
                            private_key.private_numbers().d,
                            private_key.public_key().public_numbers().n)

        if args.output:
            args.output.write(decrypted.decode())
        
        print(decrypted.decode())



In [ ]:


sys.argv = [
    "", 
    "-e", "prime",
    "-pub", "InputFiles/RSA_powmod_without_padding/public_key.pem",
    "-prv", "InputFiles/RSA_powmod_without_padding/private_key.pem",
    "-o" , "OutputFiles/q1.txt"
]
main(sys.argv[1:])

In [ ]:

with open("InputFiles/RSA_powmod_without_padding/private_key.pem", "r") as priv_key_file:
    print(priv_key_file.read())


In [ ]:
with open("InputFiles/RSA_powmod_without_padding/public_key.pem", "r") as pu_key_file:
    print(pu_key_file.read())